In [1]:
import sys
if "../src" not in sys.path:
    sys.path.append("../src")

In [2]:
import tarfile
import gzip

from tqdm.auto import tqdm
from itertools import islice
from datetime import datetime
import opencc
import textwrap

import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
output_paths = []

## Data Dependencies

```
../data/di-morphosyntax.csv 0f1c87 (LF)
```

In [3]:
from hashlib import sha1
from pathlib import Path
paths = ["../data/di-morphosyntax.csv"]
for path_x in paths:
    h = sha1()
    h.update(Path(path_x).read_bytes())
    print(path_x, h.hexdigest()[:6])

../data/di-morphosyntax.csv 0f1c87


## Load dependencies

In [4]:
t2s = opencc.OpenCC("t2s")

In [5]:
data = pd.read_csv("../data/di-morphosyntax.csv", index_col=0)
pd.value_counts(data.MorphoSyntax), pd.value_counts(data.MorphoSyntax).sum()
mr_data = data.dropna(axis=0, subset=["MorphoSyntax"])
mr_data = mr_data.assign(token_simp=[t2s.convert(x) for x in mr_data.token])
cols = [x for x in mr_data.columns.tolist() 
        if not x.startswith("token")]
cols = ["token", "token_simp"] + cols
mr_data = mr_data[cols]
print(mr_data.shape)
mr_data.head()

(1676, 9)


,token,token_simp,source,ASBC,Apple (2006-2016),China (2015-2016),Dcard (2019-2020),PTT (2004-2019),MorphoSyntax
4073,丟來,丢来,Corpus,1.0,5.0,5.0,1.0,0.0,VR
4074,丟光,丢光,Corpus,2.0,16.0,6.0,1.0,0.0,VR
4077,丟出,丢出,Corpus,9.0,190.0,56.0,16.0,0.0,VR
4079,丟到,丢到,Corpus,53.0,265.0,75.0,48.0,0.0,VR
4087,丟命,丢命,"MOE, Corpus",1.0,5.0,2.0,0.0,0.0,VO


In [6]:
mr_data.MorphoSyntax.value_counts()

VR    879
AV    340
VV    244
VO    213
Name: MorphoSyntax, dtype: int64

In [7]:
mr_data_path = "../data/verb_morpho.csv"
output_paths.append(mr_data_path)
mr_data.to_csv(mr_data_path, line_terminator="\n")

In [8]:
from itertools import chain
chars = set(list(chain(*mr_data.token)))
print("nChar, n(bisyll-)Word: ", len(chars), len(mr_data.token))

nChar, n(bisyll-)Word:  885 1676


In [9]:
gzf = gzip.open("../data/tencent-ailab-embedding-zh-d200-v0.2.0-s.tar.gz", "rb")
tarf = tarfile.open(fileobj=gzf)

words = mr_data.token.tolist() + mr_data.token_simp.tolist() + list(chars)
word_set = set(words)
tarinfo = tarf.getmember('tencent-ailab-embedding-zh-d200-v0.2.0-s/tencent-ailab-embedding-zh-d200-v0.2.0-s.txt')
embf = tarf.extractfile(tarinfo)
nvocab, ndim = [int(x) for x in 
                embf.readline().decode().strip().split()]
ln = embf.readline().decode().strip()
emb_vecs = []
vocabs = []

pbar = tqdm(total=nvocab)
while ln:
    pbar.update(1)
    toks = ln.split()    
    tok0 = toks[0]
    
    if tok0 in word_set:
        vec = [float(x) for x in toks[1:]]
        vocabs.append(tok0)
        emb_vecs.append(vec)
    ln = embf.readline().decode().strip()
pbar.close()

  0%|          | 0/2000000 [00:00<?, ?it/s]

In [10]:
kv = KeyedVectors(ndim)

In [11]:
kv.add_vectors(vocabs, emb_vecs)

In [12]:
kv.most_similar("住")

[('回', 0.562965452671051),
 ('近', 0.5430368185043335),
 ('租', 0.5377331376075745),
 ('搬去', 0.5235447287559509),
 ('破', 0.5212136507034302),
 ('定', 0.5209269523620605),
 ('起', 0.520687460899353),
 ('呆', 0.5123663544654846),
 ('到', 0.5034065842628479),
 ('去', 0.5023194551467896)]

In [13]:
kv.most_similar("泛出")

[('绽出', 0.8269344568252563),
 ('绽开', 0.7646805047988892),
 ('泛动', 0.7482767105102539),
 ('绽放', 0.7417921423912048),
 ('淌下', 0.7242736220359802),
 ('淌出', 0.7215964198112488),
 ('染满', 0.7072959542274475),
 ('殷红', 0.704120934009552),
 ('划过', 0.6990218162536621),
 ('涌动', 0.692889928817749)]

In [14]:
emb_paths = "../data/verb_morpho_emb.txt"
kv.save_word2vec_format(emb_paths)
output_paths.append(emb_paths)

## Words not found in Tencent

In [15]:
word_list = [row.token for row in mr_data.itertuples()
             if row.token not in vocabs and row.token_simp not in vocabs]
print("Not found: ", len(word_list))
print(*textwrap.wrap(" ".join(word_list), 60), sep="\n")

Not found:  613
丟到 並置 並陳 交到 交屋 交援 交溶 交磕 交絡 交纓 交著 交詮 交踏 交迸 交逼 交馳 仰戴 伸回 伸引 佈有
侵厲 修來 修到 修畢 修竣 修茂 俱長 停佇 停借 停到 停定 停慢 停棲 停租 停腳 停附 催踢 催迷 僻居 充啞
充為 充脹 充食 克奏 具帶 具瞻 兼包 兼攻 兼負 冶性 冶熔 凍成 凍斃 凍黏 刊有 刊證 剖空 剖陳 剪作 割成
割給 劃銷 劈去 勒定 勒戒 勻美 厭氣 厭身 去寒 去枝 受完 受有 受灼 受盡 叨回 叫修 叫入 叫到 叫為 叫煩
叫貨 呆藏 哄亂 哄累 售屋 售與 喊到 喊去 喊為 嚐下 嚐足 嚼出 囤壓 固接 固著 埋去 堅留 堅邀 堵回 填到
填成 填緊 填鋪 墜上 夾有 夾痛 夾織 夾蔭 夾處 奉伺 奉成 奉有 奉核 孕生 容共 寂止 寡佔 審酌 審量 屈伏
屈守 屈沈 崩倒 帶位 帶怯 帶慣 帶成 幾諫 張喙 張致 彌平 從便 忙到 思問 思算 恥言 悔斷 悔醒 悠到 悠邈
悠鳴 憨呆 憨拙 懷有 懷藏 扎來 扎完 打上 打封 打直 托來 托嬰 找沒 找無 找給 抄下 抄出 抄封 抄有 抓權
抓痛 抓硬 抓食 折拗 披瀉 披蓋 抱有 抱股 抱至 抹黃 押倒 押對 押掉 拋到 拋舉 拍取 拍妥 拓入 拓增 拜受
拜經 挖成 挖鋤 挫至 挽慰 挾到 挾得 捅去 捆死 掀走 掀頂 授獵 掉土 掠好 接成 接泊 掩出 掩泣 換掛 換搭
換為 換離 揮進 揹上 搜齊 搞來 搞早 搞禿 搞花 搞苦 搭喳 搭慣 搭曬 搭軋 摔成 摩翅 摩醞 撞扁 撞著 撥予
撥借 撥列 撥到 撥收 撥歸 撥為 撥移 撥至 撰作 撲拙 撼奮 撼樹 擇用 操成 擠回 擠榨 擠脹 擦於 擦罷 擦遍
擴佔 擴分 擴遷 攙入 攙有 攢昂 攪昏 攪盡 敘薪 敘餐 敲到 敲叩 敲桿 敲緊 斥令 斥以 施出 施動 施測 施灑
施盡 昏寐 昏脹 映演 映見 晉至 普建 普獲 普設 朗亮 朗敲 朗語 架到 架妥 架有 架造 染到 查得 查悉 查算
查讀 查贓 栓有 栓緊 栽生 梭動 梳成 檢取 檢詞 欣奮 歛容 歛盡 歛財 殷盼 決審 決行 決隄 沖失 沖清 沖融
沿傳 沿靠 泛談 泛酒 洒射 洒雨 洩除 派充 派報 派接 派援 派有 派為 派用 派請 派驗 涉於 涉有 淘到 淘掏
淪至 混有 混煮 湧到 湧生 湧至 湧退 溯入 

## Words cannot be found in tencent

In [16]:
word_list = [row.token for row in mr_data.itertuples()
             if row.token in vocabs or row.token_simp in vocabs]
print("Not found: ", len(word_list))
print(*textwrap.wrap(" ".join(word_list), 60), sep="\n")

Not found:  1063
丟來 丟光 丟出 丟命 丟擲 丟盡 丟給 丟荒 並列 並比 並稱 交下 交予 交匯 交回 交棒 交款 交納 交纏 交辦
交迭 交關 交齊 仰攻 仰翻 仰觀 仰起 估定 伸下 伸到 伸向 伸進 伸過 佈上 佈建 佈放 佈新 佈置 佈設 侵佔
侵害 侵犯 侵蝕 修下 修修 修出 修史 修完 修志 修成 修法 修滿 修短 修砌 修編 修蓋 修觀 俯察 俯望 俱存
俱生 停入 停好 停招 停掉 停播 停收 停服 停格 停步 停發 停筆 停開 偵查 催交 催催 催動 催收 催繳 催趕
催迫 充做 充入 充抵 充盈 具現 具象 兼作 兼做 兼愛 兼收 兼教 兼治 兼用 兼賣 兼顧 凍上 凍掉 凍紅 凍餓
刊於 刪掉 刪減 刷上 刷卡 刷完 剖分 剖明 剪好 剪完 剪得 剪接 剪片 剪貼 剪起 割下 割出 割完 割掉 劃上
劃下 劃亮 劃傷 劃動 劃為 劃破 劃給 劃過 劈中 劈來 劈到 劈成 劈死 劈空 劈落 勻勻 勻婷 厭勝 厭恨 去國
去邪 去鄉 受付 受僱 受勳 受報 受敵 受死 受領 叨念 叫來 叫回 叫慣 叫成 叫窮 叫進 叫道 叫響 吞入 吞吐
吞回 吞天 吞進 呆立 哄然 哄笑 售光 售完 售票 售給 唯一 唯心 喊回 喊道 喘氣 嚐出 嚐盡 嚼動 嚼嚼 嚼食
囤積 埋入 埋到 埋好 埋掉 埋置 埋進 埋頭 堅辭 堵上 堵塞 堵截 堵死 堵車 填填 填好 填膺 填進 填飽 填高
填鴨 墜崖 壞掉 壞損 壞滅 夾上 夾傷 夾出 夾到 夾峙 夾斷 夾起 夾進 奉納 奉請 容得 容有 寂照 寡歡 寡聞
審思 審校 審美 屈尊 屈屈 屈辱 屈體 崩飛 帶傷 帶到 帶大 帶好 帶引 帶淚 帶滿 帶班 帶累 帶給 帶血 帶行
帶讀 帶起 帶返 帶開 帶隊 帶離 幾經 幾近 張口 張弛 張結 張閤 從眾 忙完 忙於 恥於 悔罪 悠雅 想好 想定
想岔 想成 想盡 想破 慣看 慣稱 憑依 憑吊 憑添 憑著 憨傻 憨憨 懷柔 扎住 扎好 打低 打分 打去 打向 打回
打好 打打 打昏 打水 打火 打球 打砸 打穿 打紅 打酒 托住 托兒 托出 托大 托給 托起 扛下 扛起 扳本 扳正
扳起 找上 找來 找去 找好 找對 找找 找錯 找零 抄到 抄起 抄送 抉擇 抓下 抓來 抓傷 抓勻 抓回 抓抓 抓準
抓穩 抓舉 抓著 抓蝦 抓走 抓進 抓開 折下

## Output Hashes

```
../data/verb_morpho.csv 15c7c1
../data/verb_morpho_emb.txt f3b2a1
```

In [17]:
from hashlib import sha1
from pathlib import Path
for path_x in output_paths:
    h = sha1()
    h.update(Path(path_x).read_bytes())
    print(path_x, h.hexdigest()[:6])

../data/verb_morpho.csv 15c7c1
../data/verb_morpho_emb.txt f3b2a1
